# 🧭 GeoDashboard Skeleton v4

Interactieve verkenner voor GPKG-bestanden, met kaart, metadata en layerselectie.

In [1]:
# 📦 Widget imports (gegarandeerd bovenaan)
import ipywidgets as widgets

In [2]:
# 📦 Vereiste packages checken
import importlib
packages = ['geopandas', 'fiona', 'shapely', 'matplotlib']
for pkg in packages:
    try:
        importlib.import_module(pkg)
        print(f'✅ {pkg} geïnstalleerd')
    except ImportError:
        print(f'⚠️ {pkg} NIET geïnstalleerd')

✅ geopandas geïnstalleerd
✅ fiona geïnstalleerd
✅ shapely geïnstalleerd
✅ matplotlib geïnstalleerd


In [3]:
# 📂 Paden instellen + modules laden
import os, sys
from pathlib import Path

project_root = Path.cwd()
while not (project_root / 'config.py').exists() and project_root != project_root.parent:
    project_root = project_root.parent
sys.path.insert(0, project_root.as_posix())

from config import SRC_DIR, SHAPEFILES_DIR
if SRC_DIR.as_posix() not in sys.path:
    sys.path.insert(0, SRC_DIR.as_posix())

import geopandas as gpd
import matplotlib.pyplot as plt
from gpkg_service import list_gpkg_files, list_gpkg_layers, load_gpkg_layer

In [4]:
# 📂 GPKG-bestand selectie met alleen bestandsnamen
from gpkg_service import list_gpkg_files
from config import SHAPEFILES_DIR
gpkg_files = list_gpkg_files(SHAPEFILES_DIR)
gpkg_selector = widgets.Dropdown(
    options=[(f.name, f) for f in gpkg_files],
    description='GPKG:',
    layout=widgets.Layout(width='400px')
)

In [5]:
# 🎨 Layoutfixes en preview verbeteringen (v003h)
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import os
from config import SHAPEFILES_DIR, EXPORT_FOLDER
from gpkg_service import list_gpkg_files, list_gpkg_layers, load_gpkg_layer

plot_output = widgets.Output(layout={'justify_content': 'center'})
preview_output = widgets.Output()
export_output = widgets.Output()

gpkg_files = list_gpkg_files(SHAPEFILES_DIR)
gpkg_selector = widgets.Dropdown(
    options=[(f.name, f) for f in gpkg_files],
    description='GPKG:',
    layout=widgets.Layout(width='390px')
)

# Default 'buurten' ingesteld
layer_selector = widgets.Dropdown(
    options=['buurten'],
    description='Layer:',
    layout=widgets.Layout(width='300px')
)

sampling_dropdown = widgets.Dropdown(
    options=[('10', 0.10), ('25', 0.25), ('50', 0.50), ('100', 1.0)],
    value=1.0,
    description='Sample',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='180px')
)

export_button = widgets.Button(description='Export sample', layout=widgets.Layout(width='150px'))
title_html = widgets.HTML(value='', layout=widgets.Layout(justify_content='center'))

def update_layers(change):
    if gpkg_selector.value:
        layers = list_gpkg_layers(gpkg_selector.value)
        layer_selector.options = layers

def update_plot(*args):
    plot_output.clear_output()
    preview_output.clear_output()
    export_output.clear_output()
    try:
        gpkg_path = gpkg_selector.value
        layer = layer_selector.value
        gdf = load_gpkg_layer(gpkg_path, layer)
        frac = sampling_dropdown.value
        global sampled_gdf
        sampled_gdf = gdf.sample(frac=frac, random_state=1).reset_index(drop=True)
        title_html.value = f"<h4 style='text-align:center;'>📦 {gpkg_selector.label} | 🧭 Layer: {layer} | Sample: {int(frac*100)}%</h4><br>"
        with plot_output:
            fig, ax = plt.subplots(figsize=(6, 6))
            sampled_gdf.plot(ax=ax)
            plt.tight_layout()
            plt.show()
        with preview_output:
            print('🧾 Sample preview:')
            display(sampled_gdf.head(3))
            print(f'Shape: {sampled_gdf.shape[0]} rows × {sampled_gdf.shape[1]} columns')
    except Exception as e:
        with plot_output:
            print('⚠️ Plotfout:', e)

def export_sample(b):
    export_output.clear_output()
    try:
        fname = f"sample_{layer_selector.value}_{int(sampling_dropdown.value*100)}.geojson"
        path = os.path.join(EXPORT_FOLDER, fname)
        sampled_gdf.to_file(path, driver='GeoJSON')
        with export_output:
            print(f'✅ Sample exported to: {path}')
    except Exception as e:
        with export_output:
            print(f'❌ Export failed: {e}')

gpkg_selector.observe(update_layers, names='value')
gpkg_selector.observe(update_plot, names='value')
layer_selector.observe(update_plot, names='value')
sampling_dropdown.observe(update_plot, names='value')
export_button.on_click(export_sample)

controls = widgets.HBox([
    gpkg_selector,
    layer_selector,
    sampling_dropdown,
    export_button
])

display(widgets.VBox([
    controls,
    title_html,
    plot_output,
    widgets.HTML('<br>'),
    preview_output,
    export_output
]))

update_layers(None)
update_plot()